# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [3]:
%%writefile mapper1.py

# Your code for mapper here.

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)
        

Writing mapper1.py


In [4]:
%%writefile reducer1.py

# Your code for reducer here.
import sys

current_key = None
word_sum = 0
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)
    

Writing reducer1.py


In [15]:
%%bash
OUT_DIR="word_count_result_"$(date +"%s%6N")
NUM_REDUCERS=1

#hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Wordcount streaming" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input "/data/wiki/en_articles_part" \
    -output ${OUT_DIR} > /dev/null


18/12/19 12:52:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/19 12:52:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/19 12:52:35 INFO mapred.FileInputFormat: Total input files to process : 1
18/12/19 12:52:35 INFO mapreduce.JobSubmitter: number of splits:2
18/12/19 12:52:35 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
18/12/19 12:52:36 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1545211394186_0002
18/12/19 12:52:36 INFO impl.YarnClientImpl: Submitted application application_1545211394186_0002
18/12/19 12:52:36 INFO mapreduce.Job: The url to track the job: http://8bee9537d1bb:8088/proxy/application_1545211394186_0002/
18/12/19 12:52:36 INFO mapreduce.Job: Running job: job_1545211394186_0002
18/12/19 12:52:43 INFO mapreduce.Job: Job job_1545211394186_0002 running in uber mode : false
18/12/19 12:52:43 INFO mapreduce.Job:  map 0% reduce 0%
18/12/19 12:53:00 INFO m

In [18]:
!hdfs dfs -cat word_count_result_1545223951521647/* | head

...baby	1
0	2877
0".32	1
0%however	1
0&\mathrm{if	1
0&\mbox	1
0&\mbox{if	2
0(8)320-1231	1
0(8)320-1234	1
0)$and	1
cat: Unable to write to output stream.


In [14]:
! date

Wed Dec 19 12:51:58 UTC 2018


In [21]:
%%bash
# You can use this cell for other experiments: for example, for combiner.

OUT_DIR="word_count_result_"$(date +"%s%6N")
NUM_REDUCERS=2

#hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Wordcount streaming" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input "/data/wiki/en_articles_part" \
    -output ${OUT_DIR} > /dev/null


18/12/19 13:01:31 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/19 13:01:31 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/19 13:01:33 INFO mapred.FileInputFormat: Total input files to process : 1
18/12/19 13:01:33 INFO mapreduce.JobSubmitter: number of splits:2
18/12/19 13:01:33 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
18/12/19 13:01:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1545211394186_0003
18/12/19 13:01:34 INFO impl.YarnClientImpl: Submitted application application_1545211394186_0003
18/12/19 13:01:34 INFO mapreduce.Job: The url to track the job: http://8bee9537d1bb:8088/proxy/application_1545211394186_0003/
18/12/19 13:01:34 INFO mapreduce.Job: Running job: job_1545211394186_0003
18/12/19 13:01:41 INFO mapreduce.Job: Job job_1545211394186_0003 running in uber mode : false
18/12/19 13:01:41 INFO mapreduce.Job:  map 0% reduce 0%
18/12/19 13:01:57 INFO m

In [23]:
! hdfs dfs -ls word_count_result_1545224490005726

Found 3 items
-rw-r--r--   1 jovyan supergroup          0 2018-12-19 13:02 word_count_result_1545224490005726/_SUCCESS
-rw-r--r--   1 jovyan supergroup    2667984 2018-12-19 13:02 word_count_result_1545224490005726/part-00000
-rw-r--r--   1 jovyan supergroup    2702529 2018-12-19 13:02 word_count_result_1545224490005726/part-00001


In [22]:
! hdfs dfs -cat word_count_result_1545224490005726/* | head

0".32	1
0%however	1
0&\mathrm{if	1
0(8)320-1234	1
0)(0,0,0	1
0)).(1	2
0).euclid's	1
0).one	1
0+\|b\|^2	1
0,0,snr	1
cat: Unable to write to output stream.
cat: Unable to write to output stream.


## Step 2. Create sort job.

<b>Hint:</b> You may use MapReduce comparator to solve this step. Make sure that the keys are sorted in ascending order.

In [0]:
# Your code for sort job here. Don't forget to use magic writefile

## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [0]:
%%bash

OUT_DIR="assignment1_"$(date +"%s%6N")

# Code for your first job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...

# Code for your second job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR}/part-00000 | sed -n '7p;8q'
hdfs dfs -rm -r -skipTrash ${OUT_DIR}* > /dev/null